In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode


In [3]:



dict_creacion = {"generos" : """CREATE TABLE generos (
                                    id_genero INT AUTO_INCREMENT PRIMARY KEY,
                                    nombre_genero VARCHAR(50) NOT NULL,
                                    descripcion TEXT
                                    )""",
                "peliculas": """CREATE TABLE peliculas (
                                    id_pelicula INT AUTO_INCREMENT PRIMARY KEY,
                                    nombre VARCHAR(100) NOT NULL,
                                    duracion INT NOT NULL,
                                    direccion VARCHAR(100) NOT NULL,
                                    id_genero INT NOT NULL,
                                    fecha_estreno DATE NOT NULL,
                                    FOREIGN KEY (id_genero)
                                        REFERENCES generos(id_genero)
                                    )""",
                "clientes": """CREATE TABLE clientes (
                                    id_cliente INT AUTO_INCREMENT PRIMARY KEY,
                                    nombre VARCHAR(50) NOT NULL,
                                    apellido VARCHAR(50) NOT NULL,
                                    direccion VARCHAR(100),
                                    telefono VARCHAR(15),
                                    email VARCHAR(100),
                                    fecha_registro DATE NOT NULL
                                    )""",
                "alquileres": """CREATE TABLE alquileres (
                                    id_alquiler INT AUTO_INCREMENT PRIMARY KEY,
                                    id_cliente INT NOT NULL,
                                    id_pelicula INT NOT NULL,
                                    fecha_alquiler DATE NOT NULL,
                                    fecha_devolucion DATE,
                                    estado ENUM('pendiente', 'devuelto') NOT NULL,
                                    FOREIGN KEY (id_cliente)
                                        REFERENCES clientes(id_cliente)
                                        ON DELETE CASCADE
                                        ON UPDATE CASCADE,
                                    FOREIGN KEY (id_pelicula)
                                        REFERENCES peliculas(id_pelicula)
                                        ON DELETE CASCADE
                                        ON UPDATE CASCADE
                                )"""}


#for k, v in dict_creacion.items():
  #  print(f"estamos creando la tabla {k}")
  #  mycursor.execute(v)
  #  print("-------------------------------")


## Comenzamos creando tablas y nombrando PK y FK

In [4]:
import mysql.connector

cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',
    auth_plugin='mysql_native_password')
mycursor = cnx.cursor()
print("Conexión exitosa 🚀")

Conexión exitosa 🚀


In [6]:
mycursor.execute("CREATE SCHEMA IF NOT EXISTS videoclub")
mycursor.execute("USE videoclub")

In [7]:
for k, v in dict_creacion.items():
    print(f"estamos creando la tabla {k}")
    mycursor.execute(v)

    
    print("----------------------")

estamos creando la tabla generos
----------------------
estamos creando la tabla peliculas
----------------------
estamos creando la tabla clientes
----------------------
estamos creando la tabla alquileres
----------------------


# EJERCICIO 2 - MODIFICACIONES
Hacer una función para lanzar_query "def lanzar_query (bbdd, query)"
Tiene en cuenta esa base de datos y la consulta/query

def lanzar_query (bbdd, query):
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                host='127.0.0.1', database = bbdd
                                )
    mycursor = cnx.cursor()
    mycursor.execute(query)
    cnx.close()

query = "SELECT * FROM clientes"
lanzar_query("videoclub", "SELECT * FROM clientes")
lanzar_query("videoclub", query)

# 2. Modifica la base de datos

¡Vaya! Nos hemos equivocado en varias cosas, vamos a tener que hacer 
algunos cambios:

- Modifica la columna email de la tabla **clientes**. Ahora queremos que tenga más capacidad de caracteres y que no acepte nulos.
- Modifica la columna estado en la tabla **alquileres**. Queremos que siga aceptando solo los valores ‘pendiente’ y ‘devuelto’, pero que por  defecto el valor sea ‘pendiente’.


In [8]:
def lanzar_query (bbdd, query):
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                host='127.0.0.1', 
                                auth_plugin='mysql_native_password',
                                database = bbdd
                                )
    mycursor = cnx.cursor()
    mycursor.execute(query)
    
    cnx.close()

In [9]:
modificar_columna = """ALTER TABLE clientes
                            MODIFY COLUMN email VARCHAR(100) NOT NULL;"""

lanzar_query('videoclub', modificar_columna)

In [10]:
modificar_alquileres = """ALTER TABLE alquileres
                                MODIFY COLUMN estado ENUM ('PENDIENTE', 'DEVUELTO') DEFAULT 'PENDIENTE';"""
lanzar_query("videoclub", modificar_alquileres)

Empezamos a insertar datos en tablas del videoclub

In [ ]:
def insertar(bbdd, query, valores):
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', auth_plugin='mysql_native_password',
                              database = bbdd
                              )
    mycursor = cnx.cursor()
    mycursor.executemany(query, valores) #executemany a query y valores q aun no existen
    print(mycursor.rowcount, "registros insertados") #está en la lección
    # rowcount cuenta Cuantos registros hemos insertado
    cnx.commit()  # a la conexión,para confirmar los cambios en bbdd

    cnx.close()

La anterior def insertar no vale, necesitamos el executemany y el commit no nos deja.
REALIZAMOS LA CONSULTA/QUERY :

In [57]:
query_insert = '''
                    INSERT INTO generos (nombre_genero, descripcion)
                        VALUES (%s, %s)'''
                               


In [60]:
query_insert = '''
                    INSERT INTO peliculas (nombre, duracion, direccion, 
                    id_genero, fecha_estreno)
                        VALUES (%s, %s, %s, %s, %s)  '''

In [63]:
query_insert = '''
                    INSERT INTO clientes (nombre, apellido, direccion, 
                    telefono, email, fecha_registro)
                        VALUES (%s, %s, %s, %s, %s, %s) '''

In [78]:
query_insert = '''
                    INSERT INTO alquileres (id_cliente, id_pelicula, fecha_alquiler, 
                    fecha_devolucion, estado)
                        VALUES (%s, %s, %s, %s, %s) '''

In [58]:
generos = [('Acción', 'Películas llenas de emoción y aventuras'),
('Comedia', 'Películas para reír y disfrutar'),
('Drama', 'Películas emocionales y profundas'),
('Ciencia Ficción', 'Películas con elementos futuristas o tecnológicos')]


In [59]:
insertar("videoclub", query_insert, generos)


4 registros insertados


In [61]:
peliculas = [('Misión Rescate', 130, 'Ridley Scott', 4, '2015-10-02'), 
('La Gran Aventura', 120, 'Chris Columbus', 1, '2001-06-22'), ('Reír o Llorar', 95, 'John Smith', 2, '2020-11-12'), 
('Crisis Total', 110, 'Jane Doe', 3, '2019-05-15')]

In [62]:
insertar("videoclub", query_insert, peliculas)

4 registros insertados


In [64]:
clientes = [('Carlos', 'García', 'Av. Principal 123', '555-1234', 'carlos@example.com',  '2023-01-15'),
('Lucía', 'Pérez', 'Calle Secundaria 45', '555-5678', 'lucia@example.com',  '2023-02-20'), ('Miguel', 'Lopez', 'Av. Tercera 789', '555-9876', 'miguel@example.com', '2023-03-10')]

In [65]:
insertar("videoclub", query_insert, clientes)

3 registros insertados


In [81]:
alquileres = [(1, 1, '2025-01-01', None, 'pendiente'), (2, 2, '2025-01-03', '2025-01-07', 'devuelto'), 
(3, 4, '2025-01-05', None, 'pendiente')]

In [82]:
insertar("videoclub", query_insert, alquileres)

3 registros insertados


Funcion a la que hay que llamar para INSERTAR en bbdd

-- Ahora ejecutar una query de visualización de una tabla para imprimir un 
DataFrame de esta 
- Opción 1: mantener columnas tabla  
sql = "SELECT * FROM clientes" 
df1 = pd.read_sql_query(sql, cnx) cnx es paso2

# Tenemos un DataFrame que tenemos que cargar en este Jupyter

In [83]:
df_clientes = pd.read_csv("files/clientes.csv")

#, header=None, names=['nombre', 'apellido', 'direccion', 'telefono', 'email', 'fecha_registro'])


CUIDADO CSV es un texto plano, valores separados por comas

In [84]:
df_clientes.head()

,Miguel,Navarro,Calle Central 449,555-7961,ana@example.com,2023-02-27
0,Carmen,Navarro,Calle Norte 659,555-8199,ana@example.com,2024-10-25
1,Sofía,Ruiz,Calle Oeste 504,555-9360,ana@example.com,2023-01-10
2,Carlos,Herrera,Calle Sol 533,555-7115,jorge@example.com,2024-03-13
3,Fernando,Navarro,Calle Estrella 446,555-4390,ana@example.com,2023-04-15
4,Sofía,Pérez,Calle Central 606,555-3117,claudia@example.com,2023-03-01


In [87]:
df_clientes = pd.read_csv("files/clientes.csv", header=None)

In [88]:
df_clientes.head()


,0,1,2,3,4,5
0,Miguel,Navarro,Calle Central 449,555-7961,ana@example.com,2023-02-27
1,Carmen,Navarro,Calle Norte 659,555-8199,ana@example.com,2024-10-25
2,Sofía,Ruiz,Calle Oeste 504,555-9360,ana@example.com,2023-01-10
3,Carlos,Herrera,Calle Sol 533,555-7115,jorge@example.com,2024-03-13
4,Fernando,Navarro,Calle Estrella 446,555-4390,ana@example.com,2023-04-15


In [89]:
df_clientes = pd.read_csv("files/clientes.csv", header=None, names=['nombre', 'apellido', 'direccion', 'telefono', 'email', 'fecha_registro'])


In [90]:
df_clientes.head()

,nombre,apellido,direccion,telefono,email,fecha_registro
0,Miguel,Navarro,Calle Central 449,555-7961,ana@example.com,2023-02-27
1,Carmen,Navarro,Calle Norte 659,555-8199,ana@example.com,2024-10-25
2,Sofía,Ruiz,Calle Oeste 504,555-9360,ana@example.com,2023-01-10
3,Carlos,Herrera,Calle Sol 533,555-7115,jorge@example.com,2024-03-13
4,Fernando,Navarro,Calle Estrella 446,555-4390,ana@example.com,2023-04-15


In [ ]:
# .tail() devuelve los datos del final

In [91]:
df_clientes.tail()

,nombre,apellido,direccion,telefono,email,fecha_registro
145,Alberto,Vega,Calle Luna 947,555-8111,sofia@example.com,2023-09-05
146,Carlos,Fernández,Calle Sur 224,555-4859,pedro@example.com,2023-02-16
147,Claudia,García,Calle Oeste 690,555-9656,sofia@example.com,2024-11-14
148,Andrés,Domínguez,Calle Este 479,555-8572,elena@example.com,2023-09-06
149,Sofía,Gil,Calle Libertad 402,555-9261,maria@example.com,2023-04-27


In [93]:
# .sample() devuelve un dato aleatorio
# .sample()--- 3
df_clientes.sample(2)

,nombre,apellido,direccion,telefono,email,fecha_registro
14,Claudia,Vega,Calle Libertad 216,555-5894,sofia@example.com,2023-04-02
117,Ricardo,Gómez,Calle Sur 653,555-4393,jorge@example.com,2024-10-20


In [94]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   nombre          150 non-null    object
 1   apellido        150 non-null    object
 2   direccion       150 non-null    object
 3   telefono        150 non-null    object
 4   email           150 non-null    object
 5   fecha_registro  150 non-null    object
dtypes: object(6)
memory usage: 7.2+ KB


si fuese un dict con keys: nombres, values: valores

In [ ]:
df_clientes["nombre"].unique() #nombres únicos

array(['Miguel', 'Carmen', 'Sofía', 'Carlos', 'Fernando', 'Marta',
       'Claudia', 'Patricia', 'Pedro', 'Alberto', 'Andrés', 'Rafael',
       'Luis', 'María', 'Jorge', 'Ana', 'Elena', 'Ricardo', 'Lucía',
       'Laura'], dtype=object)

In [ ]:
#forma del df
df_clientes.shape   # 150 filas y 6 columnas

(150, 6)

In [99]:
df_clientes["nombre"]

0        Miguel
1        Carmen
2         Sofía
3        Carlos
4      Fernando
         ...   
145     Alberto
146      Carlos
147     Claudia
148      Andrés
149       Sofía
Name: nombre, Length: 150, dtype: object

TENGO QUE INSERTAR VALORES EN SQL CON UNA LISTA DE TUPLAS

o con UNA LISTA DE LISTAS

In [100]:
df_clientes.head()

,nombre,apellido,direccion,telefono,email,fecha_registro
0,Miguel,Navarro,Calle Central 449,555-7961,ana@example.com,2023-02-27
1,Carmen,Navarro,Calle Norte 659,555-8199,ana@example.com,2024-10-25
2,Sofía,Ruiz,Calle Oeste 504,555-9360,ana@example.com,2023-01-10
3,Carlos,Herrera,Calle Sol 533,555-7115,jorge@example.com,2024-03-13
4,Fernando,Navarro,Calle Estrella 446,555-4390,ana@example.com,2023-04-15


In [102]:
df_clientes.values

array([['Miguel', 'Navarro', 'Calle Central 449', '555-7961',
        'ana@example.com', '2023-02-27'],
       ['Carmen', 'Navarro', 'Calle Norte 659', '555-8199',
        'ana@example.com', '2024-10-25'],
       ['Sofía', 'Ruiz', 'Calle Oeste 504', '555-9360',
        'ana@example.com', '2023-01-10'],
       ['Carlos', 'Herrera', 'Calle Sol 533', '555-7115',
        'jorge@example.com', '2024-03-13'],
       ['Fernando', 'Navarro', 'Calle Estrella 446', '555-4390',
        'ana@example.com', '2023-04-15'],
       ['Sofía', 'Pérez', 'Calle Central 606', '555-3117',
        'claudia@example.com', '2023-03-01'],
       ['Marta', 'Domínguez', 'Calle Libertad 363', '555-9877',
        'elena@example.com', '2024-08-27'],
       ['Claudia', 'Vega', 'Calle Central 146', '555-4130',
        'luis@example.com', '2024-10-11'],
       ['Patricia', 'Castro', 'Calle Luna 821', '555-7720',
        'maria@example.com', '2024-09-02'],
       ['Pedro', 'Gómez', 'Calle Central 831', '555-9197',
        

APLICO UN METODO PARA PASAR A LISTA

In [112]:
lista_clientes = df_clientes.values.tolist()

In [113]:
lista_clientes

[['Miguel',
  'Navarro',
  'Calle Central 449',
  '555-7961',
  'ana@example.com',
  '2023-02-27'],
 ['Carmen',
  'Navarro',
  'Calle Norte 659',
  '555-8199',
  'ana@example.com',
  '2024-10-25'],
 ['Sofía',
  'Ruiz',
  'Calle Oeste 504',
  '555-9360',
  'ana@example.com',
  '2023-01-10'],
 ['Carlos',
  'Herrera',
  'Calle Sol 533',
  '555-7115',
  'jorge@example.com',
  '2024-03-13'],
 ['Fernando',
  'Navarro',
  'Calle Estrella 446',
  '555-4390',
  'ana@example.com',
  '2023-04-15'],
 ['Sofía',
  'Pérez',
  'Calle Central 606',
  '555-3117',
  'claudia@example.com',
  '2023-03-01'],
 ['Marta',
  'Domínguez',
  'Calle Libertad 363',
  '555-9877',
  'elena@example.com',
  '2024-08-27'],
 ['Claudia',
  'Vega',
  'Calle Central 146',
  '555-4130',
  'luis@example.com',
  '2024-10-11'],
 ['Patricia',
  'Castro',
  'Calle Luna 821',
  '555-7720',
  'maria@example.com',
  '2024-09-02'],
 ['Pedro',
  'Gómez',
  'Calle Central 831',
  '555-9197',
  'sofia@example.com',
  '2023-07-12'],
 ['A

In [114]:
df_clientes.itertuples()

In [120]:
for tup in df_clientes.itertuples(index=False, name=None):
    print(tup)
    break

('Miguel', 'Navarro', 'Calle Central 449', '555-7961', 'ana@example.com', '2023-02-27')


otro método: to records (+ adelante)
ITERTUPLES NOS GENERA UN OBJETO MAS FACIL DE ITERAR

In [123]:
lista_clientes = list(df_clientes.itertuples(index=False, name=None))

In [125]:
lista_tuplas_clientes = list(df_clientes.to_records(index=False))
lista_tuplas_clientes[:1]


[('Miguel', 'Navarro', 'Calle Central 449', '555-7961', 'ana@example.com', '2023-02-27')]

In [126]:
df_clientes.to_records(index=False). tolist()[:1]

[('Miguel',
  'Navarro',
  'Calle Central 449',
  '555-7961',
  'ana@example.com',
  '2023-02-27')]

In [128]:
query = 'INSERT INTO clientes (nombre, apellido, direccion, telefono, email, fecha_registro) VALUES (%s, %s, %s, %s, %s, %s)'


In [130]:
insertar('videoclub', query, lista_clientes)

150 registros insertados
